In [ ]:
# 通过uv安装依赖
! uv sync

# 下载Qwen/Qwen2.5-VL-3B-Instruct模型

In [ ]:
from modelscope import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor, snapshot_download, AutoModel, AutoTokenizer
from qwen_vl_utils import process_vision_info
model_dir = snapshot_download('qwen/Qwen2.5-VL-3B-Instruct', cache_dir='./autodl-tmp', revision='master')


2025-08-28 15:30:22,540 - modelscope - INFO - Got 2 files, start to download ...


Processing 2 items:   0%|          | 0.00/2.00 [00:00<?, ?it/s]